In [1]:
#import sys
#!{sys.executable} -m pip install datasets
from datasets import load_dataset
dataset = load_dataset('imdb')
dataset = dataset.map(lambda x: {'label': 1 if x['label'] == 'pos' else 0})

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at C:\Users\gamer\.cache\huggingface\datasets\imdb\plain_text\0.0.0\e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Wed Nov  5 09:29:07 2025).


In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub-0.31.2-py3.11.egg\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
def tokenizar(muestra):
    return tokenizer(muestra['text'], padding='max_length', truncation=True)

In [4]:
tokenized_datasets = dataset.map(tokenizar, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
from torch.utils.data import DataLoader

In [6]:
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(10000))
val_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))
train_dataloader = DataLoader(train_dataset, batch_size=8)
val_dataloader = DataLoader(val_dataset, batch_size=8)

In [7]:
from transformers import BertForSequenceClassification

In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub-0.31.2-py3.11.egg\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import AdamW

In [13]:
optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
!pip install tf-keras
!pip install transformers[torch]
from transformers import Trainer, TrainingArguments

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 1.6 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 1.7 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 1.8 MB/s  0:00:01


In [15]:
# liberar memoria
import torch
torch.cuda.empty_cache()

In [18]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer = Trainer(
    model=model,
    args=training_arg,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()